In [1]:
from glob import glob
import pandas as pd
from io import StringIO
import numpy as np
from collections import namedtuple
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
a = int("11010010",2)
b = int("11010101",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

a = int("11010010",2) - int("1101",2)
b = int("11010101",2) - int("1101",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

bin(int("11010101",2) - 
int("10101010",2))

a = int("11010010",2) - int("101011",2)
b = int("11010101",2) - int("101011",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

('0b10100100', '0b10110000')

In [3]:
num =  int("11010010",2)
pred = int("11010101",2)
center = 4
# shift_calculation(num, 32 - center, pred, domain=4)

### Shift and LZC calculation

In [4]:
def shift_calculation(num, center, pred, domain=4, bits=32):
    string_repr = np.binary_repr(pred,bits)
    right,left = min(center + domain, bits), max(center - domain, 0)
    last_bit = string_repr[left-1]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*(domain+(32-right))
    tmp = string_repr[:left]
    h = tmp+goal
    shift = int(goal,2) - pred
    return shift

In [5]:
def lzc(val, bits=32):
    """Count leading zeroes."""
    cnt = 0
    for i in range(0, bits):
        if val & (1 << (bits - 1 - i)) != 0:
            break
        cnt += 1
    return cnt
lzcu = np.frompyfunc(lzc, 2, 1)

# Test
## Read data and prepare dataframe

In [6]:
truth = np.fromfile("test.npf", "int64")

In [7]:
df = pd.DataFrame({"truth":truth})
df["prediction"] = 0

df["shifted_prediction"] = 0
df["shifted_truth"] = 0
df["shift"] = 0

df["shifted_residual"] = 0
df["normal_residual"] = 0

df["shifted_lzc"] = 0
df["normal_lzc"] = 0

df.head()

,truth,prediction,shifted_prediction,shifted_truth,shift,shifted_residual,normal_residual,shifted_lzc,normal_lzc
0,1228193487,0,0,0,0,0,0,0,0
1,1228175479,0,0,0,0,0,0,0,0
2,1228178323,0,0,0,0,0,0,0,0
3,1228204314,0,0,0,0,0,0,0,0
4,1228217687,0,0,0,0,0,0,0,0


## Calculate predictions (shifted and normal)

In [8]:
for i in range(2,df.truth.size):
    df["prediction"][i] = df["truth"][i-1]
    center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1])
    df["shift"][i] = shift_calculation(None, center,df["prediction"][i],domain=4)
    df["shifted_prediction"][i] = df["prediction"][i] + df["shift"][i]
    df["shifted_truth"][i] = df["truth"][i] + df["shift"][i]
    df["shifted_residual"][i] = df["shifted_prediction"][i] ^ df["shifted_truth"][i]
    df["normal_residual"][i] = df["prediction"][i] ^ df["truth"][i]
    df["shifted_lzc"][i] = lzc(df["shifted_prediction"][i] ^ df["shifted_truth"][i])
    df["normal_lzc"][i] = lzc(df["prediction"][i] ^ df["truth"][i])
df.head()

,truth,prediction,shifted_prediction,shifted_truth,shift,shifted_residual,normal_residual,shifted_lzc,normal_lzc
0,1228193487,0,0,0,0,0,0,0,0
1,1228175479,0,0,0,0,0,0,0,0
2,1228178323,1228175479,1431655765,1431658609,203480286,13604,64484,18,16
3,1228204314,1228178323,1431655765,1431681756,203477442,61321,27273,16,17
4,1228217687,1228204314,5726623061,5726636434,4498418747,56519,128077,16,15


## Analysis

In [9]:
analysis = [ 
   (df["normal_lzc"] <= df["shifted_lzc"]).sum(), 
   (df["shifted_lzc"]-df["normal_lzc"]).sum(), 
   df["normal_lzc"].sum(), 
   df["shifted_lzc"].sum(),
   df["normal_lzc"].sum()/(df["normal_lzc"].size*32) * 100,
   df["shifted_lzc"].sum()/(df["shifted_lzc"].size*32) * 100
]
print(
    """
    Shifted LZC >= Former LZC (of 1000): {0}
    Sum Shifted LZC vs. Sum Former LZC : {2} vs. {3} (diff: {1})
    % Shifted LZC vs. % Former LZC : {4}% vs. {5}%
    """.format(*analysis))


    Shifted LZC >= Former LZC (of 1000): 695
    Sum Shifted LZC vs. Sum Former LZC : 18032 vs. 18308 (diff: 276)
    % Shifted LZC vs. % Former LZC : 56.35% vs. 57.2125%
    


## Calculate reconstructions (shifted and normal)

In [10]:
# Reconstruction of original true value

In [11]:
df["reconstruct"] = 0
df["shifted_reconstruct"] = 0

In [12]:
for i in range(2,df.truth.size):
    center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1])
    shift = shift_calculation(None, center,df["prediction"][i],domain=4)
    shifted_truth = (df["prediction"][i] + shift) ^ (df["shifted_residual"][i] + shift) - shift
    df["shifted_reconstruct"][i] = shifted_truth - shift
    df["reconstruct"][i] = df["prediction"][i] ^ df["normal_residual"][i]

In [13]:
(np.array_equal(df["truth"][2:],df["reconstruct"][2:]),
np.array_equal(df["truth"][2:],df["shifted_reconstruct"][2:]))
# reconstruct successful?

(True, True)

In [14]:
df.tail()

,truth,prediction,shifted_prediction,shifted_truth,shift,shifted_residual,normal_residual,shifted_lzc,normal_lzc,reconstruct,shifted_reconstruct
995,1228313117,1228315766,11728124029610,11728124026961,11726895713844,2811,3691,20,20,1228313117,1228313117
996,1228312804,1228313117,366503875925,366503875612,365275562808,329,761,23,22,1228312804,1228312804
997,1228311824,1228312804,11728124029610,11728124028630,11726895716806,3196,7668,20,19,1228311824,1228311824
998,1228308125,1228311824,183251937962,183251934263,182023626138,13981,62349,18,16,1228308125,1228308125
999,1228305480,1228308125,1431655765,1431653120,203347640,7765,2773,19,20,1228305480,1228305480


## Compare with previous error addition

In [15]:
df = df[["truth","prediction"]]
df.head()

,truth,prediction
0,1228193487,0
1,1228175479,0
2,1228178323,1228175479
3,1228204314,1228178323
4,1228217687,1228204314


In [16]:
df["prev_error_prediction"] = 0

In [17]:
delta = 0
beta, parts = 1,1
for i in range(1, df["prev_error_prediction"].size):
    delta = df["truth"][i-1] - df["prediction"][i-1]
    overshot = delta < 0
    offset = np.abs(delta)
    correction = (beta * offset) / parts
    if not overshot:
        df["prev_error_prediction"][i] =  df["prediction"][i] + correction
    elif correction <= df["prediction"][i]:
        df["prev_error_prediction"][i] = df["prediction"][i] - correction
    else:
        df["prev_error_prediction"][i] = 0
        offset = 0

In [18]:
df["lzc_normal"] = lzcu(df['prediction'] ^ df['truth'], 32)
df["lzc_prev_error"] = lzcu(df['prev_error_prediction'] ^ df['truth'], 32)

In [19]:
df.head()

,truth,prediction,prev_error_prediction,lzc_normal,lzc_prev_error
0,1228193487,0,0,1,1
1,1228175479,0,1228193487,1,16
2,1228178323,1228175479,2456350958,16,0
3,1228204314,1228178323,1228181167,17,17
4,1228217687,1228204314,1228230305,15,17


In [20]:
df["lzc_normal"].mean(),df["lzc_prev_error"].mean()

(18.034, 17.946)

In [21]:
df[["lzc_normal","lzc_prev_error"]].max(axis=1).mean()

18.89

# Improve upon the shifted reconstruction

In [22]:
def shift_calculation(num, center, pred, domain=4, bits=32):
    """
    Shifted residue calculation
    """
    string_repr = np.binary_repr(pred,bits)
    right,left = min(center + domain, bits), max(center - domain, 0)
    last_bit = string_repr[left-1]
    tmp = string_repr[:left]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*((32-left)//2)
    h = tmp+goal
    if len(h) == 31:
        h = h + h[-2]
    shift = int(h,2) - pred
    return shift

In [23]:
truth = np.fromfile("test.npf", "int64")
def shifted_residual_calculation(truth, do, centeroffset):

    df = pd.DataFrame({"truth":truth})
    df["prediction"] = 0

    df["shifted_prediction"] = 0
    df["shifted_truth"] = 0
    df["shift"] = 0

    df["shifted_residual"] = 0
    df["normal_residual"] = 0

    df["shifted_lzc"] = 0
    df["normal_lzc"] = 0

    for i in range(2,df.truth.size):
        df["prediction"][i] = df["truth"][i-1]
        center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1]) + centeroffset
        df["shift"][i] = shift_calculation(None, center,df["prediction"][i],domain=do)
        df["shifted_prediction"][i] = df["prediction"][i] + df["shift"][i]
        df["shifted_truth"][i] = df["truth"][i] + df["shift"][i]
        df["shifted_residual"][i] = df["shifted_prediction"][i] ^ df["shifted_truth"][i]
        df["normal_residual"][i] = df["prediction"][i] ^ df["truth"][i]
        df["shifted_lzc"][i] = lzc(df["shifted_prediction"][i] ^ df["shifted_truth"][i])
        df["normal_lzc"][i] = lzc(df["prediction"][i] ^ df["truth"][i])
    analysis = [ 
       (df["normal_lzc"] <= df["shifted_lzc"]).sum(), 
       (df["shifted_lzc"]-df["normal_lzc"]).sum(), 
       df["normal_lzc"].sum(), 
       df["shifted_lzc"].sum(),
       df["normal_lzc"].sum()/(df["normal_lzc"].size*32) * 100,
       df["shifted_lzc"].sum()/(df["shifted_lzc"].size*32) * 100
    ]
#     print("Potential:", df[["normal_lzc","shifted_lzc"]].max(axis=1).sum())
#     print(
#         """
#         Shifted LZC >= Former LZC (of 1000): {0}
#         Sum Shifted LZC vs. Sum Former LZC : {2} vs. {3} (diff: {1})
#         % Shifted LZC vs. % Former LZC : {4}% vs. {5}%
#         """.format(*analysis))
    print("domain: {} centeroffset: {} [{}] diff:{}".format(do, centeroffset, center - do, analysis[1]))

In [24]:
for do in range(1,11):
    for centeroffset in range(-4,5):
        shifted_residual_calculation(truth, do=do, centeroffset=centeroffset)

domain: 1 centeroffset: -4 [11] diff:297
domain: 1 centeroffset: -3 [12] diff:281
domain: 1 centeroffset: -2 [13] diff:250
domain: 1 centeroffset: -1 [14] diff:218
domain: 1 centeroffset: 0 [15] diff:279
domain: 1 centeroffset: 1 [16] diff:263
domain: 1 centeroffset: 2 [17] diff:257
domain: 1 centeroffset: 3 [18] diff:207
domain: 1 centeroffset: 4 [19] diff:212
domain: 2 centeroffset: -4 [10] diff:288
domain: 2 centeroffset: -3 [11] diff:297
domain: 2 centeroffset: -2 [12] diff:281
domain: 2 centeroffset: -1 [13] diff:250
domain: 2 centeroffset: 0 [14] diff:218
domain: 2 centeroffset: 1 [15] diff:279
domain: 2 centeroffset: 2 [16] diff:263
domain: 2 centeroffset: 3 [17] diff:257
domain: 2 centeroffset: 4 [18] diff:207
domain: 3 centeroffset: -4 [9] diff:238
domain: 3 centeroffset: -3 [10] diff:288
domain: 3 centeroffset: -2 [11] diff:297
domain: 3 centeroffset: -1 [12] diff:281
domain: 3 centeroffset: 0 [13] diff:250
domain: 3 centeroffset: 1 [14] diff:218
domain: 3 centeroffset: 2 [15

Until now the shift value was dependent on the previous LZC. To better analyse the effects this will be changed to a set center

# Find out were the optimal shift position is
## One shift position for all

In [25]:
truth = np.fromfile("test.npf", "int64")
def shift_calculation_by_pos(prev_truth, prev_prev_truth, prediction, pos, bits=32):
    """
    Shifted residue calculation
    """
    string_repr = np.binary_repr(prediction,bits)
    last_bit = string_repr[pos-1]
    tmp = string_repr[:pos]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*((32-pos)//2)
    h = tmp+goal
    if len(h) == 31:
        h = h + h[-2]
    shift = int(h,2) - prediction
    return shift

In [26]:
def shifted_residual_calculation_pos(truth, shift_calculation_method, **kwargs):

    df = pd.DataFrame({"truth":truth})
    df["prediction"] = 0

    df["shifted_prediction"] = 0
    df["shifted_truth"] = 0
    df["shift"] = 0

    df["shifted_residual"] = 0
    df["normal_residual"] = 0

    df["shifted_lzc"] = 0
    df["normal_lzc"] = 0

    for i in range(2,df.truth.size):
        df["prediction"][i] = df["truth"][i-1]
        df["shift"][i] = shift_calculation_method(df["truth"][i-1], df["truth"][i-2], df["prediction"][i], **kwargs)
        df["shifted_prediction"][i] = df["prediction"][i] + df["shift"][i]
        df["shifted_truth"][i] = df["truth"][i] + df["shift"][i]
        df["shifted_residual"][i] = df["shifted_prediction"][i] ^ df["shifted_truth"][i]
        df["normal_residual"][i] = df["prediction"][i] ^ df["truth"][i]
        df["shifted_lzc"][i] = lzc(df["shifted_prediction"][i] ^ df["shifted_truth"][i])
        df["normal_lzc"][i] = lzc(df["prediction"][i] ^ df["truth"][i])
    diff = (df["shifted_lzc"]-df["normal_lzc"]).sum()
    print("pos {}, diff {}".format(pos, diff))
    return df

In [27]:
for pos in range(1,30):
    shifted_residual_calculation_pos(truth, shift_calculation_by_pos, pos=pos)

pos 1, diff 254
pos 2, diff 254
pos 3, diff 254
pos 4, diff 296
pos 5, diff 296
pos 6, diff 296
pos 7, diff 254
pos 8, diff 254
pos 9, diff 254
pos 10, diff 296
pos 11, diff 296
pos 12, diff 254
pos 13, diff 254
pos 14, diff 252
pos 15, diff 277
pos 16, diff 242
pos 17, diff 254
pos 18, diff 180
pos 19, diff 124
pos 20, diff 33
pos 21, diff 24
pos 22, diff 26
pos 23, diff 5
pos 24, diff -3
pos 25, diff 6
pos 26, diff 4
pos 27, diff 3
pos 28, diff -2
pos 29, diff -2


In [28]:
maximums = []
before = []
pos = 977
pred = truth[pos]; spred = np.binary_repr(pred,32)
tru = truth[pos+1]; stru = np.binary_repr(tru,32)
before_lcount = lzc(pred^tru)
print(spred[:before_lcount], spred[before_lcount:], lzc(truth[pos-1] ^ pred))
print(stru[:before_lcount], stru[before_lcount:])
lzcounts = [lzc((pred + shift_calculation_by_pos(None,None,pred, k))^( tru + shift_calculation_by_pos(None,None,pred, k))) for k in range(33)]
for k in range(1,33):
    shift = shift_calculation_by_pos(None,None,pred, k)
    lcount = lzc((pred + shift)^( tru + shift))
    print("{:>3} {:>32b} {:>32b} {} {}".format(k, shift, shift+tru, lcount, before_lcount))
maximums.append(max(lzcounts))
before.append(before_lcount)

010010010011011010 10101100100110 20
010010010011011010 01101011100000
  1     1100000111101010101000101111  1010101010101010100010100001111 19 18
  2     1100000111101010101000101111  1010101010101010100010100001111 19 18
  3     1100000111101010101000101111  1010101010101010100010100001111 19 18
  4        1011100111111111110000100  1001010101010101001101001100100 18 18
  5        1011100111111111110000100  1001010101010101001101001100100 18 18
  6        1011100111111111110000100  1001010101010101001101001100100 18 18
  7            111101010101000101111  1001001010101010100010100001111 19 18
  8            111101010101000101111  1001001010101010100010100001111 19 18
  9            111101010101000101111  1001001010101010100010100001111 19 18
 10            -11000000000001111100  1001001001010101001101001100100 18 18
 11            -11000000000001111100  1001001001010101001101001100100 18 18
 12               -10101010111010001  1001001001101010100010100001111 19 18
 13              

In [29]:
maximums = []
before = []
for pos in range(999):
    pred = truth[pos]; spred = np.binary_repr(pred,32)
    tru = truth[pos+1]; stru = np.binary_repr(tru,32)
    before_lcount = lzc(pred^tru)
#     print(spred[:before_lcount], spred[before_lcount:])
#     print(stru[:before_lcount], stru[before_lcount:])
    lzcounts = [lzc((pred + shift_calculation_by_pos(None,None,pred, k))^( tru + shift_calculation_by_pos(None,None,pred, k))) for k in range(33)]
#     for k in range(1,33):
#         shift = shift_calculation_by_pos(None,None,pred, k)
#         lcount = lzc((pred + shift)^( tru + shift))
#         print("{:>3} {:>32b} {:>32b} {} {}".format(k, shift, shift+tru, lcount, before_lcount))
    maximums.append(max(lzcounts))
    before.append(before_lcount)

In [30]:
(np.array(maximums)>np.array(before)).sum()

484

In [31]:
np.array(maximums).sum()/32000,np.array(before).sum()/32000

(0.59153125, 0.564)

In [32]:
np.binary_repr(truth[pos])

'1001001001101100111111010011101'

In [33]:
np.binary_repr(truth[pos+1])

'1001001001101100111010001001000'

In [34]:
(np.array(maximums)>np.array(before))

array([ True,  True, False,  True,  True, False, False,  True,  True,
       False, False, False, False,  True,  True, False, False, False,
       False,  True,  True,  True,  True, False, False, False,  True,
        True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False,  True, False,  True, False,  True,  True,
        True, False,  True, False, False,  True, False, False, False,
        True, False,  True,  True, False,  True,  True,  True, False,
       False,  True,  True, False,  True, False, False, False, False,
       False, False,  True, False,  True, False, False,  True,  True,
        True,  True, False,  True, False, False, False,  True,  True,
        True,  True, False, False, False,  True, False,  True, False,
        True, False,  True,  True, False, False,  True, False,  True,
       False,  True,  True, False, False,  True, False,  True, False,
       False,  True, False,  True,  True, False,  True,  True, False,
        True,  True,

In [35]:
zero_one = int("01"*16,2)
one_zero = int("10"*16,2)
int("01"*16,2), int("10"*16,2)

(1431655765, 2863311530)

In [36]:
cut = 20
np.binary_repr((truth[pos+1] >> cut) << cut,32)

'01001001001100000000000000000000'

In [37]:
last_value = (truth[pos+1] >> cut) & 1
last_value

1

In [38]:
def get_shift_with_overflow(num, cut):
#     assert bits in (32,64), "Unknown bits {}".format(bits)
    zero_one = 1431655765 #if bits==32 else 6148914691236517205
    one_zero = 2863311530 #if bits==32 else 12297829382473034410
    base = (num >> cut) << cut
    last_value = (num >> cut) & 1
    if last_value == 1:
        delta = zero_one >> (32 - cut)
    #     subtract = False
    #     goal = base + delta
    #     shift = num - goal
    else:
        delta = one_zero >> (32 - cut)
    #     subtract = True
    #     goal = base + delta
    #     shift = goal - num
    shift = base + delta - num
    return shift

In [39]:
def get_shift_without_overflow(num, cut, bits = 32):
    """
    Get shift value within the u32/u64 value range.
    """
    assert bits in (32,64), "Unknown bits {}".format(bits)
    zero_one = 1431655765 if bits==32 else 6148914691236517205
    one_zero = 2863311530 if bits==32 else 12297829382473034410
    base = (num >> cut) << cut
    last_value = (num >> cut) & 1
    if last_value == 1:
        delta = zero_one >> (bits - cut)
        subtract = False
        goal = base + delta
        shift = num - goal
    else:
        delta = one_zero >> (bits - cut)
        subtract = True
        goal = base + delta
        shift = goal - num
    return subtract, shift

In [40]:
output = []
for i in range(truth.size):
    o  = get_shift_with_overflow(truth[i],20) + truth[i]
    a,b = get_shift_without_overflow(truth[i],20) 
    wo = truth[i]+b if a else truth[i]-b
    output.append(o == wo)

In [41]:
def shift_value(num, sign, delta):
    return num+delta if sign else num-delta

In [42]:
pos = 48
prediction = truth[pos]
tru = truth[pos+1]

shift = get_shift_without_overflow(prediction, 20)
shifted_prediction = shift_value(prediction, *shift)
shifted_truth = shift_value(tru, *shift)

In [43]:
lzc(shifted_prediction ^ shifted_truth)

21

In [44]:
lzc(truth[pos] ^ truth[pos+1])

22

# New start

In [61]:
truth = np.fromfile("test.npf", "int64")
def shift_calculation_by_pos(prev_truth, prev_prev_truth, prediction, pos, bits=32):
    """
    Shifted residue calculation
    """
    string_repr = np.binary_repr(prediction,bits)
    last_bit = string_repr[pos-1]
    tmp = string_repr[:pos]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*((32-pos)//2)
    h = tmp+goal
    if len(h) == 31:
        h = h + h[-2]
    shift = int(h,2) - prediction
    return shift

In [62]:
truth = np.fromfile("test.npf", "int64")
def shift_calculation_to_0(prev_truth, prev_prev_truth, prediction, pos, bits=32):
    """
    Shifted residue calculation
    """
    return -prediction

In [63]:
truth = np.fromfile("test.npf", "int64")
def shift_calculation_to_1(prev_truth, prev_prev_truth, prediction, pos, bits=32):
    """
    Shifted residue calculation
    """
    return int("1"*32,2) - prediction

In [98]:
def shift_calculation_compress(prediction, truth, bits=32):
    real_residual = prediction ^ truth
    zero_residual = np.array(real_residual, dtype='uint32') + 0
    once_residual = np.array(np.invert(real_residual), dtype='uint32') + 0
    if zero_residual ^ prediction > prediction & once_residual ^ prediction > prediction:
        return once_residual    
    elif zero_residual ^ prediction < prediction & once_residual ^ prediction < prediction:
        return zero_residual
    return shift_calculation_by_pos(None, None, prediction, 20)
#     print(real_residual, zero_residual, once_residual)

In [97]:
def shift_calculation_decompress(prediction, residual, bit=32):
    inverse_residual = np.array(np.invert(residual), dtype='uint32') + 0
    if prediction ^ residual > prediction & prediction ^ inverse_residual > prediction:
        return 4294967281
    ## Hier sind wir stehen geblieben

2706

In [99]:
np.binary_repr(30, 32)

'00000000000000000000000000011110'

4294967281